In [5]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import re

In [6]:
def hani_crawling(start_date, end_date):
    eng_url_head = 'http://search.hani.co.kr/Search?command=query&keyword=kaist&media=news&submedia=&sort=d&period=all&'
    kor_url_head = 'http://search.hani.co.kr/Search?command=query&keyword=카이스트&media=news&submedia=&sort=d&period=all&'
    url_tail = 'datefrom=' + str(start_date)+ '&dateto=' + str(end_date) + "&pageseq="
    
    # 검색 결과에 따른 url 링크 저장
    url1 = []
    # url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
    page = 0
    while True:
        url_final = eng_url_head + url_tail + str(page)
        try:
            response = requests.get(url_final)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.search-none')) != 0:
                break
            elements = soup.select('.search-result-list a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_1')
        for element in elements:
            link = element.attrs['href']
            url1.append(link)
        page += 1
            
    # 검색 결과에 따른 url2 링크 저장
    url2 = []
    page = 0
    while True:
        url_final = kor_url_head + url_tail + str(page)
        try:
            response = requests.get(url_final)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.search-none')) != 0:
                break
            elements = soup.select('.search-result-list a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_22')
        for element in elements:
            link = element.attrs['href']
            url2.append(link)
        page += 1
    url = list(set(url1).union(set(url2)))
    if len(url)==0:
        print("Articles Not Found")
        return 0
    df = pd.DataFrame({"url": url}, columns=["url"])
    contents = []
    titles = []
    urls = []
    for row in tqdm(df.itertuples()):
        link = 'http:' + getattr(row, 'url')
        response = requests.get(link)
        html = response.text
        soup = bs(html,'html.parser')
        try:
            for script in soup(["script", "style"]):
                script.decompose() 
            title = soup.select('#article_view_headline')[0].get_text()
            content = soup.select('.article-text')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
            content = content.replace('\xa0', " ").replace('\u3000','').replace('\r','').replace('\n', " ")
            title = title.replace('\xa0', " ")
            title = title.replace('\n', " ")
            titles.append(title)
            contents.append(content)
            urls.append(link)
        except:
            print(row)
    df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
    df.to_excel("articles/hani.xlsx",index = False)
    df.to_pickle("pickle files/hani.pkl")

In [8]:
hani_crawling('2015.01.01', '2020.10.23')

87it [00:24,  2.40it/s]

Pandas(Index=86, url='//english.hani.co.kr/arti/english_edition/e_national/681405.html')
Pandas(Index=87, url='//www.hani.co.kr/arti/english_edition/e_national/702278.html')


101it [00:27,  3.76it/s]

Pandas(Index=101, url='//www.hani.co.kr/arti/english_edition/e_international/735208.html')


137it [00:37,  3.79it/s]

...

776it [03:12,  4.74it/s]

Pandas(Index=775, url='//www.hani.co.kr/arti/english_edition/e_business/745379.html')


869it [03:36,  2.06it/s]

Pandas(Index=868, url='//english.hani.co.kr/arti/english_edition/e_northkorea/680085.html')


910it [03:46,  4.02it/s]


In [2]:
df = pd.read_pickle('pickle files/2015.01.01-2020.10.23_hani.pkl')

In [3]:
df.to_excel('articles/2015.01.01-2020.10.23_hani.xlsx', index=False)